# Pokemon Exploration

### Note

> The Exploration of all Pokemons till **Generation 8** can be viewed in **[this](https://www.kaggle.com/swashbuckler1/pokemon-exploration-plotly)** notebook.

> **[Pokemon (Generation 1 - Generation 8)](https://www.kaggle.com/swashbuckler1/pokemon-gen1gen8)** contains the details of all Pokemons from **Generation 1** to **Generation 8**.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.graph_objects as go

In [ ]:
pokemon_data = pd.read_csv("../input/pokemon/Pokemon.csv")

In [ ]:
pokemon_data.insert(4, "Pure/Dual", pokemon_data["Type 2"].isnull().map({True: "Pure", False: "Dual"}))

## Data

**The data contains the details of all Pokemons from Generation 1 to Generation 6.**<br>

**Let's take a look at the first few rows in the data.**

In [ ]:
pokemon_data.head()

There are **800 rows** and **13 columns** in the original data. Each row corresponds to a Pokemon.<br>A new column called **"Pure/Dual"** is added for convenience.

* A **Pure** type pokemon is the one which belongs to only one type (The value for **Type 2** is **NaN**).
* A **Dual** type pokemon is the one which belongs to two types.

## You are now entering the Pokemon World....

You can take a quick look at the [Plotly Graph Instructions](https://www.kaggle.com/swashbuckler1/plotly-graph-instructions) notebook for instructions on how to interact with the different plotly graphs.

In [ ]:
pure_dual_count = pokemon_data["Pure/Dual"].value_counts()

In [ ]:
col = ["#80CBC4", "#B2FF59"]

fig = go.Figure(data=[go.Pie(labels=pure_dual_count.index,
                             values=pure_dual_count.values,
                             sort=False,
                             marker=dict(colors=col, line=dict(color='#000000', width=1.5)),
                             textinfo='label+percent',
                             textposition="inside",
                hovertemplate="Pokemon Type: %{label}<br>No. of Pokemons: %{value}<br>Proportion: %{percent}<extra></extra>"
                            )])

fig.update_layout(title={"text": "No. of Pure and Dual type Pokemons", "x": 0.5},
                  legend_title="Pokemon Type")


fig.show()

**Facts :**

* Around **48%** of the pokemons are of **Pure type** while the remaining pokemons are of **Dual type**.

<hr>

In [ ]:
gen_count = "Generation " + pokemon_data["Generation"].astype(str)
gen_count = gen_count.value_counts().sort_index()

In [ ]:
col = ["#64B5F6", "#FF5252", "#00E676", "#B388FF", "#FFB74D", "#4DB6AC"]

fig = go.Figure(data=[go.Pie(labels=gen_count.index,
                             values=gen_count.values,
                             sort=False,
                             marker=dict(colors=col, line=dict(color='#000000', width=1.5)),
                             textinfo='label+percent',
                             textposition="inside",
                hovertemplate="<b>%{label}</b><br>No. of Pokemons: %{value}<br>Proportion: %{percent}<extra></extra>"
                            )])

fig.update_layout(title={"text": "No. of Pokemons by Generation", "x": 0.5})


fig.show()

**Facts :**

* **20.8%** of all pokemons belong to **Generation 1**. The generation with **166** pokemons has the most number of pokemons among all generations.<br><br>

* **Generation 3** and **Generation 5** have more than **150** pokemons each while **Generation 2** and **Generation 4** have more than **100** pokemons each.<br><br>

* With just **82** pokemons, **Generation 6** has the least number of pokemons among all generations.<br>

<hr>

In [ ]:
leg_count = pokemon_data["Legendary"].map({False: "Non-Legendary", True: "Legendary"}).value_counts()

In [ ]:
col = ["#29B6F6", "#F44336"]

fig = go.Figure(data=[go.Pie(labels=leg_count.index,
                             values=leg_count.values,
                             sort=False,
                             marker=dict(colors=col, line=dict(color='#000000', width=1.5)),
                             textinfo='label+percent',
                             textposition="inside",
                hovertemplate="<b>%{label}</b><br>No. of Pokemons: %{value}<br>Proportion: %{percent}<extra></extra>"
                            )])

fig.update_layout(title={"text": "No. of Legendary and Non-Legendary Pokemons", "x": 0.5})


fig.show()

**Facts :**

* Only **8.13%** of all pokemons are **Legendary** while the remaining pokemons are **Non-Legendary**.

<hr>

In [ ]:
type_leg_2 = pokemon_data.groupby(["Pure/Dual", "Legendary"]).size()
type_leg_2.name = "No. of Pokemons"

type_leg_2 = type_leg_2.reset_index()
type_leg_2["Legendary"] = type_leg_2["Legendary"].map({False: "Non-Legendary", True: "Legendary"})

In [ ]:
labels = []
labels.extend(type_leg_2["Pure/Dual"].unique())
labels.extend(type_leg_2["Legendary"])

parents = []
parents.extend([""] * type_leg_2["Pure/Dual"].nunique())
parents.extend(type_leg_2["Pure/Dual"])

ids = []
ids.extend(type_leg_2["Pure/Dual"].unique())
ids.extend(type_leg_2["Pure/Dual"] + " - "  + type_leg_2["Legendary"])

In [ ]:
hovertext = [] 
hovertext.extend(
      type_leg_2.groupby("Pure/Dual")["No. of Pokemons"].sum().index + "<br>" + 
     "Total no. of Pokemons: " + type_leg_2.groupby("Pure/Dual")["No. of Pokemons"].sum().astype(str)  
                )


hovertext.extend(
    type_leg_2["Legendary"] + " (" + type_leg_2["Pure/Dual"] + ")" + "<br>" +
    "No. of Pokemons: " + type_leg_2["No. of Pokemons"].astype(str)
                )

In [ ]:
values = []
values.extend(type_leg_2.groupby("Pure/Dual")["No. of Pokemons"].sum())
values.extend(type_leg_2["No. of Pokemons"])

In [ ]:
type_col_dict = {"Pure": "#B2FF59", "Dual": "#80CBC4"}

leg_col_dict = {"Non-Legendary": "#29B6F6", "Legendary": "#F44336"}

In [ ]:
colors = np.vectorize(type_col_dict.get)(type_leg_2["Pure/Dual"].unique())

colors = colors.tolist() + type_leg_2["Legendary"].map(leg_col_dict).tolist()

In [ ]:
fig =go.Figure(go.Sunburst(
    ids=ids,
    labels=labels,
    parents=parents,
    hoverinfo="text",
    hovertext=hovertext,
    values=values,
    branchvalues="total",
    marker=dict(colors=colors, line=dict(width=1, color="#000000"))
))

fig.update_layout(title={"text": "No. of Legendary and Non-Legendary Pokemons by Type", "x": 0.5}, height=650)

fig.show()

**The graph shows the number of Pure and Dual Legendary pokemons and number of Pure and Dual Non-Legendary pokemons.** 

**Facts :**

* Out of **386** Pure type pokemons, **25** are **Legendary**.<br><br>

* Out of **414** Dual type pokemons, **40** are **Legendary**.<br>It is clear that most of the **Legendary** pokemons are **Dual** type pokemons.<br><br>

<hr>

In [ ]:
type_gen_2 = pd.pivot_table(data=pokemon_data, index="Generation", columns="Pure/Dual", values="Total", aggfunc=len)

In [ ]:
type_gen_2["Total Pokemons"] = type_gen_2.sum(axis=1)

type_gen_2["Dual %"] = (type_gen_2["Dual"]/type_gen_2["Total Pokemons"]) * 100
type_gen_2["Pure %"] = (type_gen_2["Pure"]/type_gen_2["Total Pokemons"]) * 100

type_gen_2[["Dual %", "Pure %"]] = type_gen_2[["Dual %", "Pure %"]].round(2)

In [ ]:
fig = go.Figure(data=[
    
     go.Bar(name='Dual',
           x=type_gen_2.index,
           y=type_gen_2["Dual"],
           text=type_gen_2["Total Pokemons"],
           meta=type_gen_2["Dual %"],
           hovertemplate="<b>Generation %{x}</b><br>Total no. of Pokemons: %{text}<br><br>Dual type Pokemons: %{y}<br>Dual type Rate: %{meta}%<extra></extra>",
           marker=dict(color="#80CBC4", line=dict(color='#000000', width=1.5)),
           width=[0.6]*len(type_gen_2)
           ),
    
     go.Bar(name='Pure',
           x=type_gen_2.index,
           y=type_gen_2["Pure"],
           text=type_gen_2["Total Pokemons"],
           meta=type_gen_2["Pure %"],
           hovertemplate="<b>Generation %{x}</b><br>Total no. of Pokemons: %{text}<br><br>Pure type Pokemons: %{y}<br>Pure type Rate: %{meta}%<extra></extra>",
           marker=dict(color="#B2FF59", line=dict(color='#000000', width=1.5)),
           width=[0.6]*len(type_gen_2)
           ),
    
    
])

fig.update_layout(barmode='stack')

fig.update_layout(title={"text": "No. of Pure and Dual type Pokemons by Generation", "x": 0.5},
                 xaxis_title="Generation", yaxis_title="No. of Pokemons",
                 legend_title="Pokemon Type")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)


fig.show()

**Facts :**

* **Generation 1** which has the most number of pokemons (**166**) also has the most number of Pure type pokemons (**88**).The **Pure type rate** of the generation is **53.01%** which is the highest among all generations.<br><br>

* **Generation 6** which has the least number of pokemons (**82**) also has the least number of Pure type pokemons (**32**).<br>The **Pure type rate** of the generation is **39.02%** which is the lowest among all generations.
<br><br>

* **Generation 6** also has the least number of **Dual type pokemons** (**50**) but has the highest **Dual type rate** (**60.98%**).
<br><br>

* Both **Generation 3** and **Generation 5** have **82** Dual type pokemons each which is the highest among all generations.
<br><br>

<hr>

In [ ]:
gen_leg = pd.pivot_table(data=pokemon_data, index="Generation", columns="Legendary", values="Total",  aggfunc=len)
gen_leg = gen_leg.rename(columns={False: "Non-Legendary", True: "Legendary"})

In [ ]:
fig = go.Figure(data=[
    
     go.Bar(name='Non-Legendary',
           x=gen_leg.index,
           y=gen_leg["Non-Legendary"],
           text=gen_leg.sum(axis=1),
           hovertemplate="<b>Generation %{x}</b><br>Total no. of Pokemons: %{text}<br>Non-Legendary Pokemons: %{y}<extra></extra>",
           marker=dict(color="#29B6F6", line=dict(color='#000000', width=1.5)),
           width=[0.6]*len(gen_leg)
           ),
    
     go.Bar(name='Legendary',
           x=gen_leg.index,
           y=gen_leg["Legendary"],
           text=gen_leg.sum(axis=1),
           hovertemplate="<b>Generation %{x}</b><br>Total no. of Pokemons: %{text}<br>Legendary Pokemons: %{y}<extra></extra>",
           marker=dict(color="#F44336", line=dict(color='#000000', width=1.5)),
           width=[0.6]*len(gen_leg)
           ),
    
    
])

fig.update_layout(barmode='stack')

fig.update_layout(title={"text": "Frequency of Legendary and Non-Legendary by Generation", "x": 0.5},
                 xaxis_title="Generation", yaxis_title="No. of Pokemons")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)


fig.show()

**Facts :**


* **Generation 3** which has **160** pokemons has the most number of **Legendary** pokemons (**18**) among all generations.<br><br>

* **Generation 2** which has **106** pokemons has only **5 Legendary** pokemons which is the least among all generations.<br><br>

<hr>

In [ ]:
pure_poks = pokemon_data.loc[pokemon_data["Pure/Dual"] == "Pure"].copy()

dual_poks = pokemon_data.loc[pokemon_data["Pure/Dual"] == "Dual"].copy()

In [ ]:
type_count_df = pd.DataFrame(index=np.sort(pokemon_data["Type 1"].unique()))

type_count_df["Pure"] = pure_poks["Type 1"].value_counts().sort_index()

In [ ]:
t1 = dual_poks["Type 1"].value_counts()
t2 = dual_poks["Type 2"].value_counts()

type_count_df["Dual"] = t1.add(t2)

In [ ]:
type_count_df = type_count_df.stack()

type_count_df.name = "No. of Pokemons"

type_count_df.index.set_names(["Pokemon Type", "Pure/Dual"], inplace=True)

type_count_df = type_count_df.reset_index()

In [ ]:
labels = []
labels.extend(type_count_df["Pokemon Type"].unique())
labels.extend(type_count_df["Pure/Dual"])

parents = []
parents.extend([""] * type_count_df["Pokemon Type"].nunique())
parents.extend(type_count_df["Pokemon Type"])

ids = []
ids.extend(type_count_df["Pokemon Type"].unique())
ids.extend(type_count_df["Pokemon Type"] + " - "  + type_count_df["Pure/Dual"])

In [ ]:
hovertext = [] 
hovertext.extend(
      type_count_df.groupby("Pokemon Type")["No. of Pokemons"].sum().index + "<br>" + 
     "Total no. of Pokemons: " + type_count_df.groupby("Pokemon Type")["No. of Pokemons"].sum().astype(str)  
                )


(hovertext.extend(
    type_count_df["Pokemon Type"] + " - " + type_count_df["Pure/Dual"] + "<br>" +
    "No. of Pokemons: " + type_count_df["No. of Pokemons"].astype(str)
                )

)

In [ ]:
values = []
values.extend(type_count_df.groupby("Pokemon Type")["No. of Pokemons"].sum())
values.extend(type_count_df["No. of Pokemons"])

In [ ]:
type_col_dict_2 = {"Bug": "#C0CA33", "Dark": "#795548", "Dragon": "#7E57C2", "Electric": "#FFEB3B",
                   "Fairy": "#F8BBD0", "Fighting": "#D32F2F", "Fire": "#FF3D00", "Flying": "#9FA8DA",
                   "Ghost": "#673AB7", "Grass": "#00E676", "Ground": "#FDD835", "Ice": "#84FFFF",
                   "Normal": "#D7CCC8", "Poison": "#AB47BC", "Psychic": "#FF80AB", "Rock": "#FFB74D",
                   "Steel": "#E0E0E0", "Water": "#03A9F4"}

In [ ]:
colors = np.vectorize(type_col_dict_2.get)(type_count_df["Pokemon Type"].unique())

colors = colors.tolist() + type_count_df["Pure/Dual"].map(type_col_dict).tolist()

In [ ]:
fig =go.Figure(go.Sunburst(
    ids=ids,
    labels=labels,
    parents=parents,
    hoverinfo="text",
    hovertext=hovertext,
    values=values,
    branchvalues="total",
    marker=dict(colors=colors, line=dict(width=0.5, color="#000000"))
))

fig.update_layout(title={"text": "Frequency of Pokemons by Type", "x": 0.5}, height=650)

fig.show()

**The graph shows the number of pokemons of each type.**<br>
**Additionally, for each type, the number of Pure and Dual type Pokemons is highlighted.**

**Facts:**

* Most of the pokemons belong to **Water** type followed by **Normal** type.<br>
There are **126** water type pokemons and **102** normal type pokemons.<br><br>

* The least number of pokemons belong to **Ice** type followed by **Fairy** type.<br>
There are **38** ice type pokemons and **40** fairy type pokemons.<br><br>

* The types **Water**, **Normal** and **Flying** have more than **100** pokemons each.<br><br>

* **Flying** type has the most number of **Dual** type pokemons (**99**).<br>

* **Electric** type has the least number of **Dual** type pokemons (**23**).<br><br>

* **Normal** type has the most number of **Pure** type pokemons (**61**).<br>

* **Flying** type has the least number of **Pure** type pokemons (**2**).<br><br>

<hr>

In [ ]:
type_count_df = pd.pivot_table(data=type_count_df, index="Pokemon Type", columns="Pure/Dual",
                               values="No. of Pokemons", aggfunc=sum)

In [ ]:
type_count_df["Total Pokemons"] = type_count_df.sum(axis=1)

type_count_df["Dual %"] = (type_count_df["Dual"]/type_count_df["Total Pokemons"]) * 100

type_count_df["Pure %"] = (type_count_df["Pure"]/type_count_df["Total Pokemons"]) * 100

type_count_df[["Dual %", "Pure %"]] = type_count_df[["Dual %", "Pure %"]].round(2)

In [ ]:
fig = go.Figure(data=[
    
     go.Bar(name='Dual',
           x=type_count_df.index,
           y=type_count_df["Dual"],
           text=type_count_df["Total Pokemons"],
           meta=type_count_df["Dual %"],
           hovertemplate="<b>%{x}</b><br>Total no. of Pokemons: %{text}<br><br>Dual type Pokemons: %{y}<br>Dual type Rate: %{meta}%<extra></extra>",
          marker=dict(color="#80CBC4", line=dict(color='#000000', width=1.5)),
           ),
    
    go.Bar(name='Pure',
           x=type_count_df.index,
           y=type_count_df["Pure"],
           text=type_count_df["Total Pokemons"],
           meta=type_count_df["Pure %"],
           hovertemplate="<b>%{x}</b><br>Total no. of Pokemons: %{text}<br><br>Pure type Pokemons: %{y}<br>Pure type Rate: %{meta}%<extra></extra>",
           marker=dict(color="#B2FF59", line=dict(color='#000000', width=1.5)),
          )
    
    
])

fig.update_layout(barmode='stack')

fig.update_layout(title={"text": "Frequency of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="No. of Pokemons")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)


fig.show()

**The details shown by the graph is similar to the previous graph.**<br>
**Additionally, the graph highlights the Pure type rate and Dual type rate for each Pokemon type.**

**Facts :**

* **Water** type which has the most number of pokemons has around **53%** of its pokemons belonging to **Dual** type and around **47%** of its pokemons belonging to **Pure** type.<br>

* **Ice** type which has the least number of pokemons has around **66%** of its pokemons belonging to **Dual** type and around **34%** of its pokemons belonging to **Pure** type.<br><br>

* **Flying** type which has the most number of Dual type pokemons also has the highest **Dual type rate** (**98.02%**).<br>
* **Electric** type which has the least number of Dual type pokemons has a **Dual type rate** of **46%**.<br>
* **Normal** type with **41** Dual type pokemons has the least **Dual type rate** (**40.20%**).<br><br>

* **Normal** type which has the most number of Pure type pokemons has a **Pure type rate** of **59.80%** which is the highest among all types.<br>
* **Flying** type which has the least number of Pure type pokemons also has the least **Pure type rate** (**1.98%**).<br>
* **Normal** type with **71** Pure type pokemons has the highest **Pure type rate** (**56.35%**).<br>

<hr>

In [ ]:
type_leg_pure = pd.pivot_table(data=pure_poks, index="Type 1", columns="Legendary", values="Total", aggfunc=len)
type_leg_pure = type_leg_pure.fillna(0).astype(int)

In [ ]:
t1 = pd.pivot_table(data=dual_poks, index="Type 1", columns="Legendary", values="Total", aggfunc=len)
t2 = pd.pivot_table(data=dual_poks, index="Type 2", columns="Legendary", values="Total", aggfunc=len)

type_leg_dual = t1.add(t2, fill_value=0)
type_leg_dual = type_leg_dual.fillna(0).astype(int)

In [ ]:
type_leg = type_leg_pure.add(type_leg_dual)
type_leg.index.name = "Type"

In [ ]:
type_leg.rename(columns={False: "Non-Legendary", True: "Legendary"}, inplace=True)

In [ ]:
type_leg["Total Pokemons"] = type_leg.sum(axis=1)

type_leg["Non-Legendary %"] = (type_leg["Non-Legendary"]/type_leg["Total Pokemons"]) * 100
type_leg["Legendary %"] = (type_leg["Legendary"]/type_leg["Total Pokemons"]) * 100

type_leg[["Non-Legendary %", "Legendary %"]] = type_leg[["Non-Legendary %", "Legendary %"]].round(2)

In [ ]:
fig = go.Figure(data=[
    
     go.Bar(name='Non-Legendary',
           x=type_leg.index,
           y=type_leg["Non-Legendary"],
           text=type_leg["Total Pokemons"],
           meta=type_leg["Non-Legendary %"],
           hovertemplate="<b>%{x}</b><br>Total no. of Pokemons: %{text}<br><br>Non-Legendary Pokemons: %{y}<br>Non-Legendary Rate: %{meta}%<extra></extra>",
           marker=dict(color="#29B6F6", line=dict(color='#000000', width=1.5)),
           ),
    
     go.Bar(name='Legendary',
           x=type_leg.index,
           y=type_leg["Legendary"],
           text=type_leg["Total Pokemons"],
           meta=type_leg["Legendary %"],
           hovertemplate="<b>%{x}</b><br>Total no. of Pokemons: %{text}<br><br>Legendary Pokemons: %{y}<br>Legendary Rate: %{meta}%<extra></extra>",
           marker=dict(color="#F44336", line=dict(color='#000000', width=1.5)),
           ),
    
    
])

fig.update_layout(barmode='stack')

fig.update_layout(title={"text": "No. of Legendary and Non-Legendary Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="No. of Pokemons")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)


fig.show()

**Facts :**

* **Psychic** type has the most number of **Legendary** pokemons (**19**) followed by **Dragon** type (**16**).<br>**Dragon** type has the highest **Legendary rate** (**32%**) followed by **Psychic** type (**21.11%**).
<br><br>

* **Bug** and **Poison** types have no **Legendary** pokemons.

<hr>

In [ ]:
type_gen_pure = pd.pivot_table(data=pure_poks, index="Type 1", columns="Generation", values="Total", aggfunc=len)
type_gen_pure = type_gen_pure.fillna(0).astype(int)

In [ ]:
t1 = pd.pivot_table(data=dual_poks, index="Type 1", columns="Generation", values="Total", aggfunc=len)
t2 = pd.pivot_table(data=dual_poks, index="Type 2", columns="Generation", values="Total", aggfunc=len)

type_gen_dual = t1.add(t2, fill_value=0)
type_gen_dual = type_gen_dual.fillna(0).astype(int)

In [ ]:
type_gen = type_gen_pure.add(type_gen_dual)

In [ ]:
fig = go.Figure(data=go.Heatmap(
                    z=type_gen.values,
                    x=type_gen.columns,
                    y=type_gen.index,
                    colorbar={"title": "No. of Pokemons"},
                    colorscale="dense",
                    text=type_gen_pure.values,
                    meta=type_gen_dual.values,
                    hovertemplate='<b>Generation %{x} - %{y}</b><br>Total no. of Pokemons: %{z}<br><br>Pure type Pokemons: %{text}<br>Dual type Pokemons: %{meta}<extra></extra>'))

fig.update_yaxes(autorange="reversed")

fig.update_layout(title={"text": "Frequency of Pokemons by Generation and Type", "x": 0.5},
                  xaxis_title="Generation", yaxis_title="Pokemon Type", height=600)


fig.show()

**The graph shows the number of pokemons of each Type in each Generation.**<br>
**Also, the number of Pure and Dual type pokemons in each type is highlighted.**

**Facts :**

* **Generation 1** has the most number of **Poison** type pokemons (**36**) and least number of **Dark** type pokemons (**1**).
<br><br>

* **Generation 2** has the most number of **Flying** type pokemons (**19**) and least number of **Ghost** type pokemons (**1**).
<br><br>

* **Water** type pokemons (**31**) are the most frequent in **Generation 3** while **Electric** and **Poison** type pokemons are the least frequent (**5** each).
<br><br>

* **Normal** type pokemons (**18**) are the most frequent in **Generation 4** while **Fairy** type pokemons are the least frequent (**2**).
<br><br>

* **Generation 5** has the most number of **Flying** type pokemons (**21**) and least number of **Fairy** (**3**).
<br><br>

* **Generation 6** has the most number of **Ghost** and **Grass** type pokemons (**15** each) and least number of **Ground** and **Poison** type pokemons (**2** each).
<br><br>

<hr>

In [ ]:
columns = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']

In [ ]:
attr_corr = pokemon_data[columns].corr()
attr_corr = attr_corr.round(3)

In [ ]:
fig = go.Figure(data=go.Heatmap(
                    z=attr_corr.values,
                    x=attr_corr.columns,
                    y=attr_corr.index,
                    colorbar={"title": "Correlation"},
                    colorscale="dense",
                    xgap=1.5, ygap=1.5,
                    hovertemplate='Correlation between <b>%{x}</b> and <b>%{y}</b> is <b>%{z}</b>.<extra></extra>'))

fig.update_xaxes(linecolor="black", linewidth=2, mirror=True)
fig.update_yaxes(autorange="reversed", linecolor="black", linewidth=2, mirror=True)

fig.update_layout(title={"text": "Correlation between the different attributes of a Pokemon", "x": 0.5},
                  height=600)


fig.show()

**Facts :**

* **All attributes** have significant **positive correlation** with the **Total** power of the pokemon which makes sense because the **Total** power of a pokemon is the sum of those attributes.
<br><br>

* **Attack** power, **Special Attack** power and **Special Defense** power of a pokemon are **highly correlated** with the **Total** power of the pokemon.<br>This means, a pokemon with high values in any/all of these attributes will have a high **Total** power.
<br><br>

* Among all attributes, **Speed** of a pokemon has the **lowest correlation** with **Total** power.
<br><br>

* Also, the **Defense** power of a pokemon and it's **Speed** has almost **no correlation**.

<hr>

In [ ]:
attr_stats = pd.DataFrame(index=columns, columns=["Minimum", "Minimum Pokemon", "Maximum", "Maximum Pokemon"])

In [ ]:
for col in columns:
    attr_stats.loc[col, "Minimum"] = pokemon_data[col].min()
    attr_stats.loc[col, "Minimum Pokemon"] = (
     "<br>".join(pokemon_data.loc[pokemon_data[col] == pokemon_data[col].min(), "Name"].tolist())
    )
    
    attr_stats.loc[col, "Maximum"] = pokemon_data[col].max()
    attr_stats.loc[col, "Maximum Pokemon"] = (
     "<br>".join(pokemon_data.loc[pokemon_data[col] == pokemon_data[col].max(), "Name"].tolist())
    )

In [ ]:
attr_stats.index.name = "Attribute"
attr_stats = attr_stats.reset_index()

In [ ]:
attr_stats["Attribute"] = "<b>" + attr_stats["Attribute"] + "</b>"

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=[f"<b>{col}</b>" for col in attr_stats.columns],
                line_color='darkslategray',
                fill_color='royalblue',
                font=dict(color='white'),
                align='center'),
    cells=dict(values=[attr_stats[col] for col in attr_stats.columns],
               line_color='darkslategray',
               fill_color='#E3F2FD',
               font=dict(color='black'),
               align='left'))
])

fig.update_layout(title={"text": "Statistics of the different attributes of a Pokemon", "x": 0.5}, 
                  height=650)


fig.show()

**Quick Facts :**

* **[Shedinja](https://pokemondb.net/pokedex/shedinja)** has the **least HP** (**1**) among all pokemons.<br><br>

* **[Mega Mewtwo X](https://pokemondb.net/pokedex/mewtwo)**  has the **highest Attack** power (**190**) among all pokemons.<br><br>

* **[Chansey](https://pokemondb.net/pokedex/chansey)** and **[Happiny](https://pokemondb.net/pokedex/happiny)** have the **least Defense** power (**5**) among all pokemons.<br><br>

* **[Mega Mewtwo Y](https://pokemondb.net/pokedex/mewtwo)**  has the **highest Special Attack** power (**194**) among all pokemons.<br><br>

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)**  has the **highest Special Defense** power (**230**) among all pokemons.<br><br>

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)** and **[Munchlax](https://pokemondb.net/pokedex/munchlax)** have the **least Speed** (**5**) among all pokemons.<br><br>

* **[Sunkern](https://pokemondb.net/pokedex/sunkern)** has the **least Total** power (**180**) among all pokemons.<br><br>

<hr>

In [ ]:
types = np.sort(pokemon_data["Type 1"].unique())

In [ ]:
columns = ["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed", "Total"]

In [ ]:
cols=[]
for c in columns:
    cols.append((c, f"Average {c}"))
    cols.append((c, f"Minimum {c}"))
    cols.append((c, f"Minimum {c} Pokemon"))
    cols.append((c, f"Maximum {c}"))
    cols.append((c, f"Maximum {c} Pokemon"))

In [ ]:
type_stats = pd.DataFrame(index=types, columns=pd.MultiIndex.from_tuples(cols))

In [ ]:
#@title
for col in columns:
    for i, t in enumerate(types):
        bmask = (pokemon_data["Type 1"] == t) | (pokemon_data["Type 2"] == t)
        data = pokemon_data[bmask].copy()
        type_stats.loc[t, (col, f"Average {col}")] = round(data[col].mean(), 2)
        type_stats.loc[t, (col, f"Minimum {col}")] = data[col].min()
        type_stats.loc[t, (col, f"Minimum {col} Pokemon")] = data.loc[data[col].idxmin(), "Name"]
        type_stats.loc[t, (col, f"Maximum {col}")] = data[col].max()
        type_stats.loc[t, (col, f"Maximum {col} Pokemon")] = data.loc[data[col].idxmax(), "Name"]

In [ ]:
fig = go.Figure(data=[
    
        go.Bar(name='Minimum HP', 
           x=type_stats.index, 
           y=type_stats[("HP", "Minimum HP")],
           text = type_stats[("HP", "Minimum HP Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum HP: %{y}<br>Minimum HP Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#FFFF00", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average HP', 
           x=type_stats.index, 
           y=type_stats[("HP", "Average HP")],
           hovertemplate="<b>%{x}</b><br>Average HP: %{y}<extra></extra>",
           marker=dict(color="#FF1744", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum HP', 
           x=type_stats.index, 
           y=type_stats[("HP", "Maximum HP")],
           text = type_stats[("HP", "Maximum HP Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum HP: %{y}<br>Maximum HP Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#29B6F6", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "HP statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="HP", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the HP statistics of each Pokemon Type.**<br>
**The pokemons with the Minimum HP and Maximum HP for each type are highlighted along with the Average HP of each pokemon type.**

**Facts :**

* **[Shedinja](https://pokemondb.net/pokedex/shedinja)** which is a **Bug** and **Ghost** type pokemon (**Dual** type) has the least HP (**1**) among all types of pokemons.<br>
* **[Noibat](https://pokemondb.net/pokedex/noibat)** (**Flying-Dragon** pokemon) has the minimum HP (**40**) among all **Dragon** type pokemons which is the highest when compared to the minimum HP of all other types. <br><br>

* **Dragon** type has the highest Average HP (**82.90**) followed by **Ice** type (**78.63**).<br>
* **Bug** type has the least Average HP (**56.74** HP) followed by **Poison** type (**62.60** HP).<br><br>

* **[Blissey](https://pokemondb.net/pokedex/blissey)** (**Normal** type pokemon) has the highest HP (**255**) among all types of pokemons.<br>
* **[Yanmega](https://pokemondb.net/pokedex/yanmega)** (**Bug-Flying** type pokemon) has the maximum HP (**86**) among all **Bug** type pokemons which is the least when compared to the maximum HP of all other types.

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Attack', 
           x=type_stats.index, 
           y=type_stats[("Attack", "Minimum Attack")],
           text = type_stats[("Attack", "Minimum Attack Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Attack: %{y}<br>Minimum Attack Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#FF8A65", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Attack', 
           x=type_stats.index, 
           y=type_stats[("Attack", "Average Attack")],
           hovertemplate="<b>%{x}</b><br>Average Attack: %{y}<extra></extra>",
           marker=dict(color="#0288D1", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Attack', 
           x=type_stats.index, 
           y=type_stats[("Attack", "Maximum Attack")],
           text = type_stats[("Attack", "Maximum Attack Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Attack: %{y}<br>Maximum Attack Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#69F0AE", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Attack statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="Attack", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the Attack statistics of each Pokemon Type.**<br>
**The pokemons with the Minimum Attack power and Maximum Attack power for each type are highlighted along with the Average Attack power of each pokemon type.**

**Facts :**

* **[Chansey](https://pokemondb.net/pokedex/chansey)** which is a **Normal** type pokemon has the least Attack power (**5**) among all types of pokemons.<br>
* **[Purrloin](https://pokemondb.net/pokedex/purrloin)** has the minimum Attack power (**50**) among all **Dark** type pokemons which is the highest when compared to the minimum Attack power of all other types.<br><br>

* **Dragon** type has the highest Average Attack power (**105.76**) followed by **Fighting** type (**104.66**).<br>
* **Fairy** type has the least Average Attack power (**61.58**) followed by **Electric** type (**69.52**).<br><br>

* **[Mega Abomasnow](https://pokemondb.net/pokedex/abomasnow)** (**Grass-Ice** type pokemon) has the maximum Attack power (**132**) among all **Grass** type pokemons which is the lowest when compared to the maximum Attack power of all other types.<br>

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Defense', 
           x=type_stats.index, 
           y=type_stats[("Defense", "Minimum Defense")],
           text = type_stats[("Defense", "Minimum Defense Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Defense: %{y}<br>Minimum Defense Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#BCBABE", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Defense', 
           x=type_stats.index, 
           y=type_stats[("Defense", "Average Defense")],
           hovertemplate="<b>%{x}</b><br>Average Defense: %{y}<extra></extra>",
           marker=dict(color="#A1D6E2", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Defense', 
           x=type_stats.index, 
           y=type_stats[("Defense", "Maximum Defense")],
           text = type_stats[("Defense", "Maximum Defense Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Defense: %{y}<br>Maximum Defense Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#1995AD", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Defense statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="Defense", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the Defense statistics of each Pokemon Type.**<br>
**The pokemons with the Minimum Defense power and Maximum Defense power for each type are highlighted along with the Average Defense power of each pokemon type.**

**Facts :**

* **[Chansey](https://pokemondb.net/pokedex/chansey)** which is a **Normal** type pokemon has the least Defense power (**5**) among all types of pokemons.<br>
* **[Aegislash Blade Forme](https://pokemondb.net/pokedex/aegislash)** (**Steel-Ghost** type pokemon) has the minimum Defense power (**50**) among all **Steel** type pokemons which is the highest when compared to the minimum Defense power of all other types.<br><br>

* **Steel** type has the highest Average Defense power (**116.61**) followed by **Rock** type (**107.09**).<br>
* **Normal** type has the least Average Defense power (**59.61**) followed by **Poison** type (**63.74**).<br><br>

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)** (**Bug-Rock** type pokemon) and **[Mega Steelix](https://pokemondb.net/pokedex/steelix)** (**Steel-Ground** type pokemon) have the joint-highest Defense power (**230**) among all types of pokemons.<br>
* **[Zekrom](https://pokemondb.net/pokedex/zekrom)** (**Dragon-Electric** type pokemon) has the maximum Defense power (**120**) among all **Electric** type pokemons which is the lowest when compared to the maximum Defense power of all other types.<br>

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Special Attack', 
           x=type_stats.index, 
           y=type_stats[("Sp. Atk", "Minimum Sp. Atk")],
           text = type_stats[("Sp. Atk", "Minimum Sp. Atk Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Special Attack: %{y}<br>Minimum Special Attack. Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#BDBDBD", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Special Attack', 
           x=type_stats.index, 
           y=type_stats[("Sp. Atk", "Average Sp. Atk")],
           hovertemplate="<b>%{x}</b><br>Average Special Attack: %{y}<extra></extra>",
           marker=dict(color="#4DB6AC", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Special Attack', 
           x=type_stats.index, 
           y=type_stats[("Sp. Atk", "Maximum Sp. Atk")],
           text = type_stats[("Sp. Atk", "Maximum Sp. Atk Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Special Attack: %{y}<br>Maximum Special Attack Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#FBC02D", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Special Attack statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="Special Attack", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the Special Attack statistics of each Pokemon Type.**<br>
**The pokemons with the Minimum Special Attack power and Maximum Special Attack power for each type are highlighted along with the Average Special Attack power of each pokemon type.**

**Facts :**

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)** (**Bug-Rock** type pokemon) and **[Feebas](https://pokemondb.net/pokedex/feebas)** (**Water** type pokemon) have the least Special Attack power (**10**) among all types of pokemons.<br><br>

* **[Pichu](https://pokemondb.net/pokedex/pichu)** has the minimum Special Attack power (**35**) among all **Electric** type pokemons which is the highest when compared to the minimum Special Attack power of all other types.<br><br>

* **Dragon** type has the highest Average Special Attack power (**97.44**) followed by **Psychic** type (**94.60**).<br>
* **Bug** type has the least Average Special Attack power (**53.57**) followed by **Normal** type (**57.08**).<br><br>

* **[Mega Mewtwo Y](https://pokemondb.net/pokedex/mewtwo)** (**Psychic** type pokemon) has the highest Special Attack power (**194**) among all types of pokemons.<br>
* **[Volcarona](https://pokemondb.net/pokedex/volcarona)** (**Bug-Fire** type pokemon) has the maximum Special Attack power (**135**) among all **Bug** type pokemons which is the lowest when compared to the maximum Special Attack power of all other types.<br>
**Also, [Mega Pidgeot](https://pokemondb.net/pokedex/pidgeot)** (**Normal-Flying** type pokemon) has the highest Special Attack power (**194**) among all **Flying** type pokemons which is the lowest when compared to the maximum Special Attack power of all other types..<br>

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Special Defense', 
           x=type_stats.index, 
           y=type_stats[("Sp. Def", "Minimum Sp. Def")],
           text = type_stats[("Sp. Def", "Minimum Sp. Def Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Special Defense: %{y}<br>Minimum Special Defense Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#651FFF", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Special Defense', 
           x=type_stats.index, 
           y=type_stats[("Sp. Def", "Average Sp. Def")],
           hovertemplate="<b>%{x}</b><br>Average Special Defense: %{y}<extra></extra>",
           marker=dict(color="#00E676", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Special Defense', 
           x=type_stats.index, 
           y=type_stats[("Sp. Def", "Maximum Sp. Def")],
           text = type_stats[("Sp. Def", "Maximum Sp. Def Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Special Defense: %{y}<br>Maximum Special Defense Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#FF9800", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Special Defense statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="HP", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the Special Defense statistics of each Pokemon Type.**<br>
**The pokemons with the Minimum Special Defense power and Maximum Special Defense power for each type are highlighted along with the Average Special Defense power of each pokemon type.**

**Facts :**

* **[Caterpie](https://pokemondb.net/pokedex/caterpie)** (**Bug** type pokemon), **[Weedle](https://pokemondb.net/pokedex/weedle)** (**Bug-Poison** type pokemon), **[Igglybuff](https://pokemondb.net/pokedex/igglybuff)** (**Normal-Fairy** type pokemon), **[Carvanha](https://pokemondb.net/pokedex/carvanha)** (**Water-Dark** type pokemon) **[Magikarp](https://pokemondb.net/pokedex/magikarp)** (**water** type pokemon) and **[Deoxys Attack Forme](https://pokemondb.net/pokedex/deoxys)** (**Psychic** type pokemon) have the least Special Defense power (**20**) among all types of pokemons.<br>
* **[Slugma](https://pokemondb.net/pokedex/slugma)** has the minimum Special Defense power (**40**) among all **Fire** type pokemons which is the highest when compared to the minimum Special Defense power of all other types.<br><br>

* **Dragon** type has the highest Average Special Defense power (**86.90**) followed by **Psychic** type (**86.77**).<br>
* **Normal** type has the least Average Special Defense power (**63.75**) followed by **Bug** type (**64.67**).<br><br>

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)** (**Bug-Rock** type pokemon) has the highest Special Defense power (**230**) among all types of pokemons.<br>
* **[Mega Ampharos](https://pokemondb.net/pokedex/ampharos)** (**Electric-Dragon** type pokemon) has the maximum Special Defense power (**110**) among all **Electric** type pokemons which is the lowest when compared to the maximum Special Defense power of all other types.<br>

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Speed', 
           x=type_stats.index, 
           y=type_stats[("Speed", "Minimum Speed")],
           text = type_stats[("Speed", "Minimum Speed Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Speed: %{y}<br>Minimum Speed Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#FF7043", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Speed', 
           x=type_stats.index, 
           y=type_stats[("Speed", "Average Speed")],
           hovertemplate="<b>%{x}</b><br>Average Speed: %{y}<extra></extra>",
           marker=dict(color="#4FC3F7", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Speed', 
           x=type_stats.index, 
           y=type_stats[("Speed", "Maximum Speed")],
           text = type_stats[("Speed", "Maximum Speed Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Speed: %{y}<br>Maximum Speed Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#9E9E9E", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Speed statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="Speed", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the statistics of Speed of each Pokemon Type.**<br>
**The pokemons with the Minimum Speed and Maximum Speed for each type are highlighted along with the Average Speed of each pokemon type.**

**Facts :**

* **[Shuckle](https://pokemondb.net/pokedex/shuckle)** (**Bug-Rock** type pokemon), and **[Munchlax](https://pokemondb.net/pokedex/munchlax)** (**Normal** type pokemon) have the least Speed (**5**) among all types of pokemons.<br>
* **[Togetic](https://pokemondb.net/pokedex/togetic)** (**Fairy-Flying** type pokemon) has the minimum Speed (**40**) among all **Flying** type pokemons which is the highest when compared to the minimum Speed of all other types.<br><br>

* **Flying** type has the highest Average Speed (**86.39**) followed by **Electric** type (**82.94**).<br>
* **Rock** type has the least Average Speed (**51.10**) followed by **Fairy** type (**55.85**).<br><br>

* **[Deoxys Speed Forme](https://pokemondb.net/pokedex/deoxys)** (**Psychic** type pokemon) has the highest Speed (**180**) among all types of pokemons.<br>
* **[Mega Lucario](https://pokemondb.net/pokedex/lucario)** (**Fighting-Steel** type pokemon) has the maximum Speed (**112**) among all **Steel** type pokemons which is the lowest when compared to the maximum Speed of all other types.<br>

<hr>

In [ ]:
fig = go.Figure(data=[
        go.Bar(name='Minimum Total', 
           x=type_stats.index, 
           y=type_stats[("Total", "Minimum Total")],
           text = type_stats[("Total", "Minimum Total Pokemon")],
           hovertemplate="<b>%{x}</b><br>Minimum Total: %{y}<br>Minimum Total Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#00E676", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Average Total', 
           x=type_stats.index, 
           y=type_stats[("Total", "Average Total")],
           hovertemplate="<b>%{x}</b><br>Average Total: %{y}<extra></extra>",
           marker=dict(color="#FF5252", line=dict(color='#000000', width=1.5))
          ),
    
    go.Bar(name='Maximum Total', 
           x=type_stats.index, 
           y=type_stats[("Total", "Maximum Total")],
           text = type_stats[("Total", "Maximum Total Pokemon")],
           hovertemplate="<b>%{x}</b><br>Maximum Total: %{y}<br>Maximum Total Pokemon: <b>%{text}</b><extra></extra>",
           marker=dict(color="#40C4FF", line=dict(color='#000000', width=1.5))
          )
])

fig.update_layout(barmode='group', height=600)

fig.update_layout(title={"text": "Total statistics of Pokemons by Type", "x": 0.5},
                 xaxis_title="Pokemon Type", yaxis_title="Total", hovermode="x")

fig.update_xaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True,
                 ticks="outside", tickson="boundaries", ticklen=10, tickangle = 45)
fig.update_yaxes(showgrid=False, linecolor="black", linewidth=2, mirror=True)

fig.show()

**The graph shows the statistics of Total power  of each Pokemon Type.**<br>
**The pokemons with the Minimum Total power and Maximum Total power for each type are highlighted along with the Average Total power of each pokemon type.**

**Facts :**

* **[Sunkern](https://pokemondb.net/pokedex/sunkern)** (**Grass** type pokemon) has the least Total power (**180**) among all types of pokemons.<br>
* **[Beldum](https://pokemondb.net/pokedex/beldum)** (**Steel-Psychic** type pokemon) has the minimum Total power (**300**) among all **Steel** type pokemons which is the highest when compared to the minimum Total power of all other types.<br><br>

* **Dragon** type has the highest Average Total power (**541.76**) followed by **Steel** type (**486.59**).<br>
* **Bug** type has the least Average Total power (**379.53**) followed by **Poison** type (**397.69**).<br><br>

* **[Mega Mewtwo X](https://pokemondb.net/pokedex/mewtwo)** (**Psychic-Fighting** type pokemon) and **[Mega Rayquaza](https://pokemondb.net/pokedex/rayquaza)** (**Dragon-Flying** type pokemon) have the joint-highest Total power (**780**) among all types of pokemons.<br>
* **[Mega Pinsir](https://pokemondb.net/pokedex/pinsir)** (**Bug-Flying** type pokemon) has the maximum Total power (**600**) among all **Bug** type pokemons which is the lowest when compared to the maximum Total power of all other types.<br>

<hr>

In [ ]:
def stat_func(data):
    s = pd.Series(index=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=["Attribute", ""]),
              dtype=int)
    for col in columns:
        s.loc[(col, "Minimum")] =  data[col].min()
        s.loc[(col, "Minimum Pokemon")] = data.loc[data[col].idxmin(), "Name"]
        s.loc[(col, "Average")] = round(data[col].mean(),2)
        s.loc[(col, "Maximum")] = data[col].max()
        s.loc[(col, "Maximum Pokemon")] = data.loc[data[col].idxmax(), "Name"]
        
    return s

In [ ]:
leg_stats = pokemon_data.groupby(pokemon_data["Legendary"].map({False: "Non-Legendary", True: "Legendary"})).apply(stat_func)

In [ ]:
leg_stats = leg_stats.stack(level=0)

In [ ]:
leg_stats = leg_stats.reset_index()

In [ ]:
labels = []
labels.extend(leg_stats["Legendary"].unique())
labels.extend(leg_stats["Attribute"])

In [ ]:
parents = []
parents.extend([""] * leg_stats["Legendary"].nunique())
parents.extend(leg_stats["Legendary"].astype(str))

In [ ]:
ids = []
ids.extend(leg_stats["Legendary"].unique())
ids.extend(leg_stats["Legendary"] + " - " + leg_stats["Attribute"])

In [ ]:
hovertext = []
hovertext.extend(leg_stats["Legendary"].unique())

text = (
        "<b>" + leg_stats["Legendary"] + "</b>" + "<br><br>" +
        "Minimum " + leg_stats["Attribute"] + ": " + leg_stats["Minimum"].astype(str) + "<br>" + 
        "Minimum " + leg_stats["Attribute"] +  " Pokemon: " + leg_stats["Minimum Pokemon"] +  "<br>" +
        "Average " + leg_stats["Attribute"] + ": " + leg_stats["Average"].astype(str) + "<br>" +
        "Maximum " + leg_stats["Attribute"] + ": " + leg_stats["Maximum"].astype(str) + "<br>" +
        "Maximum " + leg_stats["Attribute"] +  " Pokemon: " + leg_stats["Maximum Pokemon"]
)

hovertext.extend(text)

In [ ]:
colors = ["#F44336", "#29B6F6"]

In [ ]:
fig = go.Figure(go.Treemap(
    ids=ids,
    labels = labels,
    parents = parents,
    hoverinfo="text",
    hovertext=hovertext,
    marker=dict(colors=colors, line=dict(width=1, color="#000000"))
    ))

fig.update_layout(title={"text": "Statistics of Legendary and Non-Legendary Pokemons", "x": 0.5, "y": 0.89})

fig.show()

**The graph shows the statistics of the different attributes of Legendary and Non-Legendary Pokemons.**

**Facts :**

* **Among all Legendary pokemons :**<br><br>

    * **[Deoxys Normal Forme](https://pokemondb.net/pokedex/deoxys)** has the lowest **HP** (**50**).<br><br>

    * **[Deoxys Attack Forme](https://pokemondb.net/pokedex/deoxys)** has the lowest **Defense** power (**20**) and **Special Defense** power (**20**) while **[Regirock](https://pokemondb.net/pokedex/regirock)** has the lowest **Speed** (**50**).<br><br>

    * **[Giratina Altered Forme](https://pokemondb.net/pokedex/giratina)** has the highest **HP**.<br><br>

    * **[Mega Mewtwo X](https://pokemondb.net/pokedex/mewtwo)** has the highest **Attack** power (**190**), **[Mega Mewtwo Y](https://pokemondb.net/pokedex/mewtwo)** has the highest **Special Attack** power (**194**) and **[Deoxys Speed Forme](https://pokemondb.net/pokedex/deoxys)** has the highest **Speed** (**200**).<br><br><br>
    
* **Among all Non-Legendary pokemons :**<br><br>

    * **[Shedinja](https://pokemondb.net/pokedex/shedinja)** has the lowest **HP** (**1**) while **[Blissey](https://pokemondb.net/pokedex/blissey)** has the highest **HP** (**255**).<br><br>

    * **[Chansey](https://pokemondb.net/pokedex/chansey)** has the lowest **Attack** power (**5**) and **Defense** power (**5**) while **[Mega Heracross](https://pokemondb.net/pokedex/heracross)** has the highest **Attack** power (**185**) and **[Mega Steelix](https://pokemondb.net/pokedex/steelix)** has the highest **Defense** power (**230**).<br><br>

    * **[Shuckle](https://pokemondb.net/pokedex/shuckle)** has the lowest **Special Attack** power (**10**), **[Caterpie](https://pokemondb.net/pokedex/caterpie)** has the lowest **Special Defense** power (**20**) while **[Mega Alakazam](https://pokemondb.net/pokedex/alakazam)** has the highest **Special Attack** power (**175**) and **[Shuckle](https://pokemondb.net/pokedex/shuckle)** has the highest **Special Defense** power (**230**).<br><br>

    * **[Shuckle](https://pokemondb.net/pokedex/shuckle)** has the lowest **Speed** (**5**), **[Sunkern](https://pokemondb.net/pokedex/sunkern)** has the lowest **Total** power (**180**) while **[Ninjask](https://pokemondb.net/pokedex/ninjask)** has the highest **Speed** (**160**) and<br>**[Mega Tyranitar](https://pokemondb.net/pokedex/tyranitar)** has the highest **Total** power (**700**).<br><br>

<hr>

In [ ]:
gen_stats = pokemon_data.groupby("Generation").apply(stat_func)

In [ ]:
gen_stats = gen_stats.stack(level=0)

In [ ]:
gen_stats = gen_stats.reset_index()

In [ ]:
labels = []
labels.extend(np.vectorize(lambda x: f"Generation {x}")(gen_stats["Generation"].unique()))
labels.extend(gen_stats["Attribute"])

In [ ]:
parents = []
parents.extend([""] * gen_stats["Generation"].nunique())
parents.extend("Generation " + gen_stats["Generation"].astype(str))

In [ ]:
ids = []
ids.extend(np.vectorize(lambda x: f"Generation {x}")(gen_stats["Generation"].unique()))
ids.extend("Generation " + gen_stats["Generation"].astype(str) + " - " + gen_stats["Attribute"])

In [ ]:
hovertext = []
hovertext.extend(np.vectorize(lambda x: f"Generation {x}")(gen_stats["Generation"].unique()))

text = (
        "<b>" + "Generation " + gen_stats["Generation"].astype(str) + "</b>" + "<br><br>" +
        "Minimum " + gen_stats["Attribute"] + ": " + gen_stats["Minimum"].astype(str) + "<br>" + 
        "Minimum " + gen_stats["Attribute"] +  " Pokemon: " + gen_stats["Minimum Pokemon"] +  "<br>" +
        "Average " + gen_stats["Attribute"] + ": " + gen_stats["Average"].astype(str) + "<br>" +
        "Maximum " + gen_stats["Attribute"] + ": " + gen_stats["Maximum"].astype(str) + "<br>" +
        "Maximum " + gen_stats["Attribute"] +  " Pokemon: " + gen_stats["Maximum Pokemon"]
    
)

hovertext.extend(text)

In [ ]:
col = ["#64B5F6", "#FF5252", "#00E676", "#B388FF", "#FFB74D", "#4DB6AC"]

In [ ]:
fig = go.Figure(go.Treemap(
    ids=ids,
    labels = labels,
    parents = parents,
    hoverinfo="text",
    hovertext=hovertext,
    marker=dict(colors=col, line=dict(width=1, color="#000000"))
    ))
fig.update_layout(title={"text": "Statistics of Pokemons by Generation", "x": 0.5, "y": 0.9}, 
                  height=600)

fig.show()

**The graph shows the statistics of the different attributes of Pokemons in each Generation.**

**Quick Facts :**

* In **Generation 1**, **[Mega Slowbro](https://pokemondb.net/pokedex/slowbro)** has the highest **Defense** power (**180**) while **[Mega Alakazam](https://pokemondb.net/pokedex/alakazam)** has the highest **Speed** (**150**).<br><br>

* In **Generation 2**, **[Mega Heracross](https://pokemondb.net/pokedex/heracross)** has the highest **Attack** power (**185**) while **[Mega Tyranitar](https://pokemondb.net/pokedex/tyranitar)** has the highest **Total** power (**700**).<br><br>

* **[Primal Kyogre](https://pokemondb.net/pokedex/kyogre)** has the highest **Special Attack** power (**180**) while **[Regice](https://pokemondb.net/pokedex/regice)** has the highest **Special Defense** power (**200**) in **Generation 3**.<br><br>

* **[Drifblim](https://pokemondb.net/pokedex/drifblim)** has the highest **HP** (**150**) while **[Arceus](https://pokemondb.net/pokedex/arceus)** has the highest **Total** power (**720**) in **Generation 4**.<br><br>

* **[Black Kyurem](https://pokemondb.net/pokedex/kyurem)** has the highest **Attack** power (**170**) and **Total** power (**700**) while **[Cryogonal](https://pokemondb.net/pokedex/cryogonal)** has the highest **Special Defense** power (**135**) in<br>**Generation 5**.<br><br>

* In **Generation 6**, **[Xerneas](https://pokemondb.net/pokedex/xerneas)** has the highest **HP** (**126**) while **[Talonflame](https://pokemondb.net/pokedex/talonflame)** has the highest **Speed** (**126**).<br><br>

<hr>

In [ ]:
gen_leg_stats = pokemon_data.groupby(by=["Generation", "Legendary"]).apply(stat_func).stack(level=0)

In [ ]:
gen_leg_stats = gen_leg_stats.reset_index()

In [ ]:
gen_leg_stats["Generation"] = "Generation " + gen_leg_stats["Generation"].astype(str)

gen_leg_stats["Legendary"] = gen_leg_stats["Legendary"].map({False: "Non-Legendary", True: "Legendary"})

In [ ]:
lab = []
lab.extend(gen_leg_stats["Generation"].unique())
lab.extend(gen_leg_stats["Legendary"].unique().tolist() * gen_leg_stats["Generation"].nunique())
lab.extend(gen_leg_stats["Attribute"])

In [ ]:
par = []
par.extend([""] * gen_leg_stats["Generation"].nunique())
par.extend([gen for gen in gen_leg_stats["Generation"].unique() for i in range(gen_leg_stats["Legendary"].nunique())])
par.extend(gen_leg_stats["Generation"] + " - " + gen_leg_stats["Legendary"])

In [ ]:
ids = []
ids.extend(gen_leg_stats["Generation"].unique())
ids.extend([f"{gen} - {leg}" for gen in gen_leg_stats["Generation"].unique() for leg in gen_leg_stats["Legendary"].unique()])
ids.extend(gen_leg_stats["Generation"] + " - " + gen_leg_stats["Legendary"] + " - " + gen_leg_stats["Attribute"])

In [ ]:
hovertext = []
hovertext.extend(gen_leg_stats["Generation"].unique())
hovertext.extend(gen_leg_stats["Legendary"].unique().tolist() * gen_leg_stats["Generation"].nunique())

text = (
        "<b>" + gen_leg_stats["Generation"] + " - " + gen_leg_stats["Legendary"] + "</b><br><br>" + 
        "Minimum " + gen_leg_stats["Attribute"] + ": " + gen_leg_stats["Minimum"].astype(str) + "<br>" + 
        "Minimum " + gen_leg_stats["Attribute"] + " Pokemon: " + gen_leg_stats["Minimum Pokemon"] + "<br>" +
        "Average " + gen_leg_stats["Attribute"] + ": " + gen_leg_stats["Average"].astype(str) + "<br>" + 
        "Maximum " + gen_leg_stats["Attribute"] + ": " + gen_leg_stats["Maximum"].astype(str) + "<br>" +
        "Maximum " + gen_leg_stats["Attribute"] + " Pokemon: " + gen_leg_stats["Maximum Pokemon"]
        )

hovertext.extend(text)

In [ ]:
fig = go.Figure(go.Treemap(
    ids=ids,
    labels=lab,
    parents=par,
    hoverinfo="text",
    hovertext=hovertext,
    marker=dict(colors=col, line=dict(width=1, color="#000000"))
    ))

fig.update_layout(title={"text": "Statistics of Legendary and Non-Legendary Pokemons by Generation", "x": 0.5, "y": 0.89})

fig.show()

**The graph shows the statistics of the different attributes of Legendary and Non-Legendary Pokemons in each Generation.**

**Quick Facts :**

* **Generation 1**
<br>

    * **[Mega Gyarados](https://pokemondb.net/pokedex/gyarados)** has the highest **Special Defense** power (**130**) and **Total** power (**640**) among all **Non-Legendary** pokemons while **[Articuno](https://pokemondb.net/pokedex/articuno)** has the highest **Special Defense** power (**125**) and **[Mega Mewtwo X](https://pokemondb.net/pokedex/mewtwo)** has the highest **Total** power (**780**) among all **Legendary** pokemons.
<br><br>
    
* **Generation 2**
<br>

    * **[Crobat](https://pokemondb.net/pokedex/crobat)** has the highest **Speed** (**130**) among all **Non-Legendary** pokemons while **[Raikou](https://pokemondb.net/pokedex/raikou)** has the highest **Speed** (**115**) among all<br>**Legendary** pokemons.
<br><br>

* **Generation 3**
<br>

    * **[Mega Banette](https://pokemondb.net/pokedex/banette)** has the highest **Attack** power (**165**) among all **Non-Legendary** pokemons while **[Primal Groudon](https://pokemondb.net/pokedex/groudon)** has the highest **Attack** power (**180**) among all **Legendary** pokemons.
<br><br>

* **Generation 4**
<br>

    * **[Mega Lucario](https://pokemondb.net/pokedex/lucario)** has the highest **Special Attack** power (**140**) among all **Non-Legendary** pokemons while **[Dialga](https://pokemondb.net/pokedex/dialga)** has the highest **Special Attack** power (**150**) among all **Legendary** pokemons.
<br><br>

* **Generation 5**
<br>

    * **[Cofagrigus](https://pokemondb.net/pokedex/cofagrigus)** has the highest **Defense** power (**145**) among all **Non-Legendary** pokemons while **[Cobalion](https://pokemondb.net/pokedex/cobalion)** has the highest **Defense** power (**129**) among all **Legendary** pokemons.
<br><br>
    
* **Generation 6**
<br>

    * **[Gogoat](https://pokemondb.net/pokedex/gogoat)** has the highest **HP** (**123**) among all **Non-Legendary** pokemons while **[Xerneas](https://pokemondb.net/pokedex/xerneas)** has the highest **HP** (**126**) among all **Legendary** pokemons.
<br><br>


<hr>

## Hope you enjoyed the Exploration. Thank You!!